<a href="https://colab.research.google.com/github/adipai/statistical-data-pruning-analysis/blob/main/random-pruning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sdv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.8/169.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of sdmetrics to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.7/170.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82

In [2]:
!pip install pmlb

In [3]:
# All imports here
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from pmlb import fetch_data
import matplotlib.pyplot as plt
import seaborn as sns
import random

from sdv.datasets.local import load_csvs
from sdv.metadata import SingleTableMetadata
from sdv.single_table import GaussianCopulaSynthesizer
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix

## Data preprocessing

In [4]:
# Generic data pre-processing

def preprocess_data_train(df):

    # Count missing values before handling missing data
    missing_before = df.isnull().sum().sum()
    print("Number of missing values before handling:", missing_before)

    # Handle missing data
    imputer = SimpleImputer(strategy='mean')  # You can change the strategy as needed
    df[df.select_dtypes(include=['float64', 'int64']).columns] = imputer.fit_transform(df.select_dtypes(include=['float64', 'int64']))

    # Count missing values after handling missing data
    missing_after = df.isnull().sum().sum()
    print("Number of missing values after handling:", missing_after)

    # Normalize numeric columns
    scaler = StandardScaler()
    df[df.select_dtypes(include=['float64', 'int64']).columns] = scaler.fit_transform(df.select_dtypes(include=['float64', 'int64']))

    return df, scaler, imputer

def preprocess_data_test(df, scaler, imputer):
    # Count missing values before handling missing data
    missing_before = df.isnull().sum().sum()
    print("Number of missing values before handling in test_dataset:", missing_before)

    # Handle missing data
    df[df.select_dtypes(include=['float64', 'int64']).columns] = imputer.transform(df.select_dtypes(include=['float64', 'int64']))

    # Count missing values after handling missing data
    missing_after = df.isnull().sum().sum()
    print("Number of missing values after handling in test_dataset:", missing_after)

    # Normalize numeric columns
    df[df.select_dtypes(include=['float64', 'int64']).columns] = scaler.transform(df.select_dtypes(include=['float64', 'int64']))

    return df

## Experiments

### Dataset 1: Breast cancer

In [5]:
breast_cancer = fetch_data('breast_cancer')
breast_cancer.describe()

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,target
count,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000
mean,2.664336,1.073427,4.062937,1.073427,1.167832,2.048951,0.468531,2.772727,0.237762,0.297203
std,1.011818,0.986680,2.151187,1.935321,0.443052,0.738217,0.499883,1.099006,0.426459,0.457828
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,3.000000,0.000000,1.000000,2.000000,0.000000,2.000000,0.000000,0.000000
50%,3.000000,2.000000,4.000000,0.000000,1.000000,2.000000,0.000000,3.000000,0.000000,0.000000
75%,3.000000,2.000000,5.000000,1.000000,1.000000,3.000000,1.000000,3.000000,0.000000,1.000000
max,5.000000,2.000000,10.000000,6.000000,2.000000,3.000000,1.000000,5.000000,1.000000,1.000000


## Training and testing using ML models

In [6]:
# Generic function to test synthetic data using LR, SVM, DT

def evaluate_models(X_train, X_test, y_train, y_test):

    # Initialize classifiers
    classifiers = {
        "Logistic Regression": LogisticRegression(),
        "SVM": SVC(),
        "Decision Tree": DecisionTreeClassifier()
    }

    # Results dictionary to store evaluation metrics
    results = {}

    # Iterate over classifiers
    for name, clf in classifiers.items():
        # Fit classifier
        clf.fit(X_train, y_train)

        # Predictions
        y_pred = clf.predict(X_test)

        # Evaluation metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        # AUC-ROC
        if hasattr(clf, "predict_proba"):
            y_prob = clf.predict_proba(X_test)[:,1]
        else:
            y_prob = clf.decision_function(X_test)
        fpr, tpr, thresholds = roc_curve(y_test, y_prob)
        roc_auc = auc(fpr, tpr)

        # Confusion matrix
        cm = confusion_matrix(y_test, y_pred)

        # Store results
        results[name] = {
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1 Score": f1,
            "ROC AUC": roc_auc,
            "Confusion Matrix": cm
        }

        # Plot AUC-ROC curve
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'{name} - AUC-ROC Curve')
        plt.legend(loc='lower right')
        plt.savefig(f'{name}_auc_roc_curve.png', dpi=300)
        plt.close()

        # Plot confusion matrix
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title(f'{name} - Confusion Matrix')
        plt.savefig(f'{name}_confusion_matrix.png', dpi=300)
        plt.close()

    return results


In [7]:

y = breast_cancer['target']
X = breast_cancer.drop('target', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

preprocessed_X_train, scaler, imputer = preprocess_data_train(X_train)
preprocessed_X_test = preprocess_data_test(X_test, scaler, imputer)

X_train, y_train = preprocessed_X_train.to_numpy(), y_train.to_numpy()
X_test, y_test = preprocessed_X_test.to_numpy(), y_test.to_numpy()


Number of missing values before handling: 0
Number of missing values after handling: 0
Number of missing values before handling in test_dataset: 0
Number of missing values after handling in test_dataset: 0


In [ ]:
type(X_train), type(y_train)

(numpy.ndarray, numpy.ndarray)

In [8]:
import time

"""
inputs:
X: np.array
y: np.array
percentage: from 0% upto 100%, enter int value
"""
def prune_data(X, y, percentage, seed = 42):
  np.random.seed(seed)
  labels_count = {}
  labels = np.unique(y)
  for label in labels:
    labels_count[label] = np.count_nonzero(y == label)
  max_label = min_label = labels[0]
  for label in labels_count:
    if labels_count[label] > labels_count[max_label]:
      max_label = label
    if labels_count[label] < labels_count[min_label]:
      min_label = label

  # print("Max", max_label, labels_count[max_label])
  # print("Min", min_label, labels_count[min_label])

  prune_counts = {}
  prune_indexes = {}
  for label in labels_count:
    prune_counts[label] = labels_count[label] - labels_count[min_label]
    prune_indexes[label] = np.where(y == label)[0]

  prune_amount = int((percentage / 100) * sum(map(lambda x: x[1], prune_counts.items())))
  prune_it = {}

  while prune_amount > 0:
    for label in labels:
      if (len(prune_indexes[label]) - labels_count[min_label]) > 0 and prune_amount > 0:
        random_index = np.random.choice(len(prune_indexes[label]))
        random_item = prune_indexes[label][random_index]
        prune_indexes[label] = np.delete(prune_indexes[label], random_index)
        if prune_it.get(label, None) is None:
          prune_it[label] = np.array([])
        prune_it[label] = np.append(prune_it[label], [random_item])
        prune_amount -= 1



  formatted_indexes = np.array([])
  for label in prune_indexes:
    formatted_indexes = np.append(formatted_indexes, prune_indexes[label])
  formatted_indexes = np.sort(formatted_indexes)
  new_arr = np.array([np.int64(i) for i in formatted_indexes])

  return X[new_arr], y[new_arr]

In [9]:
X_train_0, y_train_0 =  prune_data(X_train, y_train, 0)
evaluate_models(X_train_0, X_test, y_train_0, y_test)

{'Logistic Regression': {'Accuracy': 0.7931034482758621,
  'Precision': 0.8,
  'Recall': 0.4444444444444444,
  'F1 Score': 0.5714285714285714,
  'ROC AUC': 0.7291666666666667,
  'Confusion Matrix': array([[38,  2],
         [10,  8]])},
 'SVM': {'Accuracy': 0.7758620689655172,
  'Precision': 0.7777777777777778,
  'Recall': 0.3888888888888889,
  'F1 Score': 0.5185185185185185,
  'ROC AUC': 0.7833333333333333,
  'Confusion Matrix': array([[38,  2],
         [11,  7]])},
 'Decision Tree': {'Accuracy': 0.6551724137931034,
  'Precision': 0.45454545454545453,
  'Recall': 0.5555555555555556,
  'F1 Score': 0.5,
  'ROC AUC': 0.6666666666666666,
  'Confusion Matrix': array([[28, 12],
         [ 8, 10]])}}

In [10]:
X_train_20, y_train_20 =  prune_data(X_train, y_train, 20)
evaluate_models(X_train_20, X_test, y_train_20, y_test)

{'Logistic Regression': {'Accuracy': 0.7931034482758621,
  'Precision': 0.8,
  'Recall': 0.4444444444444444,
  'F1 Score': 0.5714285714285714,
  'ROC AUC': 0.7236111111111111,
  'Confusion Matrix': array([[38,  2],
         [10,  8]])},
 'SVM': {'Accuracy': 0.7586206896551724,
  'Precision': 0.7,
  'Recall': 0.3888888888888889,
  'F1 Score': 0.5,
  'ROC AUC': 0.8027777777777778,
  'Confusion Matrix': array([[37,  3],
         [11,  7]])},
 'Decision Tree': {'Accuracy': 0.7413793103448276,
  'Precision': 0.5789473684210527,
  'Recall': 0.6111111111111112,
  'F1 Score': 0.5945945945945946,
  'ROC AUC': 0.7055555555555556,
  'Confusion Matrix': array([[32,  8],
         [ 7, 11]])}}

In [11]:
X_train_40, y_train_40 =  prune_data(X_train, y_train, 40)
evaluate_models(X_train_40, X_test, y_train_40, y_test)

{'Logistic Regression': {'Accuracy': 0.7758620689655172,
  'Precision': 0.7272727272727273,
  'Recall': 0.4444444444444444,
  'F1 Score': 0.5517241379310345,
  'ROC AUC': 0.7194444444444446,
  'Confusion Matrix': array([[37,  3],
         [10,  8]])},
 'SVM': {'Accuracy': 0.7758620689655172,
  'Precision': 0.7777777777777778,
  'Recall': 0.3888888888888889,
  'F1 Score': 0.5185185185185185,
  'ROC AUC': 0.8097222222222222,
  'Confusion Matrix': array([[38,  2],
         [11,  7]])},
 'Decision Tree': {'Accuracy': 0.7068965517241379,
  'Precision': 0.5263157894736842,
  'Recall': 0.5555555555555556,
  'F1 Score': 0.5405405405405405,
  'ROC AUC': 0.6597222222222222,
  'Confusion Matrix': array([[31,  9],
         [ 8, 10]])}}

In [12]:
X_train_60, y_train_60 =  prune_data(X_train, y_train, 60)
evaluate_models(X_train_60, X_test, y_train_60, y_test)

{'Logistic Regression': {'Accuracy': 0.7758620689655172,
  'Precision': 0.7272727272727273,
  'Recall': 0.4444444444444444,
  'F1 Score': 0.5517241379310345,
  'ROC AUC': 0.673611111111111,
  'Confusion Matrix': array([[37,  3],
         [10,  8]])},
 'SVM': {'Accuracy': 0.7586206896551724,
  'Precision': 0.6666666666666666,
  'Recall': 0.4444444444444444,
  'F1 Score': 0.5333333333333333,
  'ROC AUC': 0.8055555555555556,
  'Confusion Matrix': array([[36,  4],
         [10,  8]])},
 'Decision Tree': {'Accuracy': 0.6724137931034483,
  'Precision': 0.47619047619047616,
  'Recall': 0.5555555555555556,
  'F1 Score': 0.5128205128205129,
  'ROC AUC': 0.6402777777777777,
  'Confusion Matrix': array([[29, 11],
         [ 8, 10]])}}

In [13]:
X_train_80, y_train_80 =  prune_data(X_train, y_train, 80)
evaluate_models(X_train_80, X_test, y_train_80, y_test)

{'Logistic Regression': {'Accuracy': 0.7758620689655172,
  'Precision': 0.6666666666666666,
  'Recall': 0.5555555555555556,
  'F1 Score': 0.606060606060606,
  'ROC AUC': 0.6583333333333333,
  'Confusion Matrix': array([[35,  5],
         [ 8, 10]])},
 'SVM': {'Accuracy': 0.7413793103448276,
  'Precision': 0.6153846153846154,
  'Recall': 0.4444444444444444,
  'F1 Score': 0.5161290322580646,
  'ROC AUC': 0.7944444444444445,
  'Confusion Matrix': array([[35,  5],
         [10,  8]])},
 'Decision Tree': {'Accuracy': 0.7586206896551724,
  'Precision': 0.6,
  'Recall': 0.6666666666666666,
  'F1 Score': 0.631578947368421,
  'ROC AUC': 0.7333333333333333,
  'Confusion Matrix': array([[32,  8],
         [ 6, 12]])}}

In [16]:
X_train_100, y_train_100 =  prune_data(X_train, y_train, 100)
evaluate_models(X_train_100, X_test, y_train_100, y_test)

{'Logistic Regression': {'Accuracy': 0.6724137931034483,
  'Precision': 0.4782608695652174,
  'Recall': 0.6111111111111112,
  'F1 Score': 0.5365853658536586,
  'ROC AUC': 0.6722222222222223,
  'Confusion Matrix': array([[28, 12],
         [ 7, 11]])},
 'SVM': {'Accuracy': 0.7241379310344828,
  'Precision': 0.55,
  'Recall': 0.6111111111111112,
  'F1 Score': 0.5789473684210527,
  'ROC AUC': 0.788888888888889,
  'Confusion Matrix': array([[31,  9],
         [ 7, 11]])},
 'Decision Tree': {'Accuracy': 0.6379310344827587,
  'Precision': 0.4444444444444444,
  'Recall': 0.6666666666666666,
  'F1 Score': 0.5333333333333333,
  'ROC AUC': 0.6416666666666666,
  'Confusion Matrix': array([[25, 15],
         [ 6, 12]])}}